In [ ]:
import cv2
import mediapipe as mp
mp_hands = mp.solutions.hands
mp_face_mesh = mp.solutions.face_mesh
mp_pose = mp.solutions.pose
hands = mp_hands.Hands(static_image_mode=False, 
                       max_num_hands=2, 
                       min_detection_confidence=0.5, 
                       min_tracking_confidence=0.5)
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=False, 
                                  max_num_faces=1, 
                                  min_detection_confidence=0.5, 
                                  min_tracking_confidence=0.5)
pose = mp_pose.Pose(static_image_mode=False,
                    min_detection_confidence=0.5,
                    min_tracking_confidence=0.5)

filtered_hand = list(range(21))
filtered_pose = [0, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]
filtered_face = [10, 152, 14, 7, 8, 144, 145, 146, 153, 154, 155, 158, 159, 160, 161, 163, 362, 385, 387, 388, 390, 398, 400, 402, 417, 419, 420, 421, 423, 424, 425, 426, 427, 428, 429, 430, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449]
# Constants for padding
POSE_LANDMARKS = len(filtered_pose)
FACE_LANDMARKS = len(filtered_face)
HAND_LANDMARKS = len(filtered_hand)
TOTAL_LANDMARKS = POSE_LANDMARKS + FACE_LANDMARKS + (2 * HAND_LANDMARKS)
print(TOTAL_LANDMARKS)
def extract_landmarks_from_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        hand_results = hands.process(image_rgb)
        face_results = face_mesh.process(image_rgb)
        pose_results = pose.process(image_rgb)

        if face_results.multi_face_landmarks:
            for face_landmarks in face_results.multi_face_landmarks:
                for id in filtered_face:
                    landmark = face_landmarks.landmark[id]
                    h, w, _ = frame.shape
                    cx, cy = int(landmark.x * w), int(landmark.y * h)
                    cv2.circle(frame, (cx, cy), 2, (255, 0, 0), -1) 

        if hand_results.multi_hand_landmarks:
            for hand_landmarks in hand_results.multi_hand_landmarks:
                for id in filtered_hand:
                    landmark = hand_landmarks.landmark[id]
                    h, w, _ = frame.shape
                    cx, cy = int(landmark.x * w), int(landmark.y * h)
                    cv2.circle(frame, (cx, cy), 2, (0, 255, 0), -1)  

        if pose_results.pose_landmarks:
            for id in filtered_pose:
                landmark = pose_results.pose_landmarks.landmark[id]
                h, w, _ = frame.shape
                cx, cy = int(landmark.x * w), int(landmark.y * h)
                cv2.circle(frame, (cx, cy), 2, (0, 0, 255), -1) 

        out.write(frame)

    cap.release()
    out.release()

extract_landmarks_from_video('MVI_5177.MOV', 'output_video.mp4')


111
